In [1]:
#%pip install pyserial

In [16]:
import serial

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.signal
%matplotlib qt

In [18]:
fs = 200000

In [19]:
def tomaLecturas(puerto:str,cantidad:int,ancho:int=2,orden:str='little'):
    def procesaEscape(s:bytes):
        def proceso(s:bytes):
            i = 0
            while i < len(s):
                sk = s[i]
                match sk:
                    case b'\x0c':
                        pass
                    case b'\xdb':
                        if i+1 == len(s): break
                        match s[i+1]:
                            case b'\xdc':
                                yield b'\x0c'
                            case b'\xdd':
                                yield b'\xdb'
                            case _:
                                break
                    case _:
                        yield sk
        return bytes(proceso(s))
    def recibeBytes(puerto,cantidad):
        separador = b'\x0c'
        def generadorAuxiliar(puerto,cantidad):
            with serial.Serial(puerto) as s:
                cuenta = 0
                s.read_until(separador)
                while cuenta < cantidad:
                    datos=s.read_until(separador)
                    if len(datos)<=1 : datos = s.read_until(separador)
                    datos = procesaEscape(datos)
                    yield from datos
                    cuenta -= len(datos)
        return bytes(generadorAuxiliar(puerto,cantidad))
        
    datos = recibeBytes(puerto,cantidad*ancho)
    cantidad = len(datos)//ancho
    return np.array([ int.from_bytes(datos[ancho*i:ancho*(i+1)],orden,signed=False) for i in range(cantidad)])

In [20]:
lecturas = tomaLecturas("/dev/ttyACM0",fs)

KeyboardInterrupt: 

In [14]:
plt.plot(lecturas)


In [15]:
plt.stem(*scipy.signal.periodogram(lecturas,fs,'hamming'))

<StemContainer object of 3 artists>

In [58]:
with serial.Serial("/dev/ttyACM0") as s:
    d = s.read(1024)

In [59]:
d.find(b'\x0c\x0c',511)

-1

In [62]:
d[127+127:].find(b'\x0c')

127

In [66]:
d[127+125:]

b'\x07\x0c9\x08\xee\x07\\\x08\xfc\x07l\x08\x13\x08\xce\x07D\x08\xf4\x07f\x08\x00\x08\xc4\x07;\x08\xf0\x07b\x08\x05\x08\xc2\x07?\x08\xec\x07^\x08\xff\x07n\x08\xdb\xdc\x08\xce\x07E\x08\xef\x07`\x08\xfe\x07o\x08\x11\x08\xd0\x07E\x08\xf2\x07\\\x08\t\x08\xc2\x07;\x08\xf0\x07_\x08\x01\x08\xc5\x07<\x08\xea\x07c\x08\x07\x08\xc5\x07<\x08\xf3\x07^\x08\x05\x08\xc5\x07@\x08\xee\x07]\x08\xff\x07j\x08\x0e\x08\xcb\x07=\x08\xf5\x07a\x08\x02\x08\xc7\x07\x0c>\x08\xf2\x07`\x08\x06\x08\xc1\x07<\x08\xf1\x07b\x08\x06\x08\xc5\x07>\x08\xef\x07X\x08\xff\x07r\x08\x0e\x08\xd0\x07F\x08\xf3\x07^\x08\x08\x08\xc4\x07@\x08\xe9\x07X\x08\xfd\x07m\x08\x08\x08\xc9\x07A\x08\xea\x07V\x08\xff\x07k\x08\xdb\xdc\x08\xcc\x07>\x08\xf5\x07_\x08\x07\x08\xc5\x07C\x08\xe9\x07V\x08\x03\x08\xbf\x07>\x08\xed\x07_\x08\x02\x08\xc0\x07@\x08\xe6\x07V\x08\x00\x08\xba\x07@\x08\xef\x07W\x08\x01\x08\xc2\x07;\x08\xeb\x07\x0c_\x08\x02\x08\xc3\x07;\x08\xec\x07\\\x08\x02\x08\xc2\x07>\x08\xf0\x07X\x08\x07\x08\xc0\x07:\x08\xeb\x07[\x08\xff\x07i\x08\